In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyspark
!pip install pmdarima
!pip install darts

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
import datetime
import warnings
warnings.simplefilter("ignore")

# from datetime import datetime
sc = SparkContext('local')


In [ ]:
spark = SparkSession(sc)
df=spark.read.format("csv").option("header","true").load('/kaggle/input/yahoo-finance-stock-market-data/stock_market_data.csv')
df.show()

In [ ]:
# warnings.simplefilter("ignore")
# df = df.withColumn("Date", F.to_date(F.col("Date"), 'dd/MM/yyyy'))
# min_date = df.select(F.min("Date")).first()[0]
# max_date = df.select(F.max("Date")).first()[0]
# date_range = [min_date + datetime.timedelta(days=x) for x in range((max_date - min_date).days + 1)]
# all_dates_df = spark.createDataFrame([(date,) for date in date_range], ["Date"])
# full_df = all_dates_df.join(df, on="Date", how="left_outer").orderBy("Date")
# window = (Window.orderBy('Date').rowsBetween(Window.unboundedPreceding, Window.currentRow))
# full_df = (full_df.withColumn('AMZN', F.last('AMZN', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('GOOG', F.last('GOOG', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('QCOM', F.last('QCOM', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('AMAT', F.last('AMAT', ignorenulls=True).over(window)))
    
# full_df.show()


In [ ]:
# full_df.toPandas().to_csv('pre_processed_data.csv',header=True)
spark.stop()

In [ ]:
stock_data = pd.read_csv('/kaggle/input/yahoo-finance-stock-market-data/pre_processed_data.csv')
stock_data = stock_data.drop(columns=['Unnamed: 0'])
stock_data.head()

**THE SHARE PRICES ARE IN POUNDS AND NOT IN DOLLARS**

In [ ]:
import plotly.express as px
fig = px.line(stock_data, x='Date', y=["GOOG","AMZN","QCOM","AMAT"],title='Stock market data')
fig.show()

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set 'Date' as the index
stock_data.set_index('Date', inplace=True)

# Resample data bi-weekly and calculate the mean for each group
_12_weekly_averages = stock_data.resample('12W').mean()

print('Length of Original data',len(stock_data))
print('Length of 12 weekly data',len(_12_weekly_averages))
# Display the first few rows of the bi-weekly average data
_12_weekly_averages.head()

In [ ]:
_12_weekly_averages_df = _12_weekly_averages.reset_index()
fig = px.line(_12_weekly_averages_df, x='Date', y=["GOOG","AMZN","QCOM","AMAT"],title='Stock market data')
fig.show()

# **Autoregressive (AR) model**
An Autoregressive (AR) model is a type of time series forecasting model that uses a linear combination of the past values of the variable to predict future values. The model can be represented as:


Xt
​
 =c+ϕ 
1
​
 Xt−1
​
 +ϕ 
2
​
 Xt−2
​
 +…+ϕ 
p
​
 Xt−p
​
 +ϵt
​
 

Here, 

Xt
​
is the variable at time 

t,c is a constant, 

ϕ1
​
 ,ϕ2
​
 ,…,ϕp are parameters of the model, and ϵt is white noise.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

# Function to fit AR model and forecast the next period
def forecast_ar(stock_data):
    # Fit an AR model
    model = ARIMA(stock_data, order=(1,0,0))  # Equivalent to AR(1) model
    model_fit = model.fit()
    # Forecast the next bi-weekly period
    forecast = model_fit.forecast(steps=1)
    
    # Fit an AR model
    model = ARIMA(stock_data[:-1], order=(1,0,0))  # Equivalent to AR(1) model
    model_fit = model.fit()
    # Forecast the next bi-weekly period
    last_forecast = model_fit.forecast(steps=1)
    
    return model_fit, forecast,last_forecast[0],stock_data[-1]

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}
df_results=pd.DataFrame(columns=['Model','Company','Actual','Forecasted','Error'])
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, forecast,last_forecast,actual = forecast_ar(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, forecast)
    models_last_forecasts[column]=(fit,[actual,last_forecast,actual-last_forecast])
    
    df= pd.DataFrame([['AR',column,actual,last_forecast,actual-last_forecast]],columns=['Model','Company','Actual','Forecasted','Error'])
    df_results=pd.concat([df_results,df])
    df_results=df_results.reset_index(drop=True)
# # Display the forecasted last values for each stock   
# print({key: val[1] for key, val in models_last_forecasts.items()})
# Display the forecasted future values for each stock
print('prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})
df_results.head()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_ar_forecast_comparison(row, col, stock_name):
    # Fit AR(1) model excluding the last bi-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(1,0,0))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_ar_forecast_comparison(1, 1, 'AMZN')
plot_ar_forecast_comparison(1, 2, 'GOOG')
plot_ar_forecast_comparison(2, 1, 'QCOM')
plot_ar_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="AR MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **Moving Average (MA)**
A Moving Average (MA) model in time series forecasting is a method that models the next time step as a function of the past error terms. The formula for an MA model of order \( q \) is given by:

\[ X_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \ldots + \theta_q \epsilon_{t-q} \]

where \( \mu \) is the mean of the series, \( \epsilon_t \) is white noise, and \( \theta_1, \theta_2, \ldots, \theta_q \) are the parameters of the model. This approach is particularly useful for modeling time series that show short-term dependencies between consecutive observations.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Function to fit MA model and forecast the next period
def forecast_ma(stock_data):
    # Fit an MA model for future prediction
    model_future = ARIMA(stock_data, order=(0,0,1))  # Equivalent to MA(1) model
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit an MA model excluding the last data point for last period prediction
    model_last = ARIMA(stock_data[:-1], order=(0,0,1))  # Equivalent to MA(1) model
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_ma(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['MA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])
    
df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)
# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='MA'].head()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_ma_forecast_comparison(row, col, stock_name):
    # Fit MA(1) model excluding the last 12-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(0,0,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_ma_forecast_comparison(1, 1, 'AMZN')
plot_ma_forecast_comparison(1, 2, 'GOOG')
plot_ma_forecast_comparison(2, 1, 'QCOM')
plot_ma_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="MA MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **ARIMA (AutoRegressive Integrated Moving Average)**
An ARIMA (AutoRegressive Integrated Moving Average) model is used for forecasting time series data by combining differencing (to make data stationary), autoregression (AR), and moving average (MA) components. The formula for an ARIMA model is generally expressed as:

\[ \nabla^d X_t = c + \phi_1 \nabla^d X_{t-1} + \dots + \phi_p \nabla^d X_{t-p} + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q} + \epsilon_t \]

where \( \nabla^d X_t \) represents the differenced series, \( p \) and \( q \) are the orders of the AR and MA parts, \( d \) is the degree of differencing, \( \phi \) and \( \theta \) are parameters, and \( \epsilon_t \) is white noise. This model can capture complex patterns in time series data, making it effective for a wide range of applications.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Function to fit ARIMA model and forecast the next period
def forecast_arima(stock_data):
    # Fit an ARIMA model for future prediction
    model_future = ARIMA(stock_data, order=(1,1,1))  # ARIMA(1,1,1) model
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit an ARIMA model excluding the last data point for last period prediction
    model_last = ARIMA(stock_data[:-1], order=(1,1,1))  # ARIMA(1,1,1) model
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_arima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['ARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='ARIMA'].head()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_arima_forecast_comparison(row, col, stock_name):
    # Fit ARIMA(1,1,1) model excluding the last 12-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(1,1,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_arima_forecast_comparison(1, 1, 'AMZN')
plot_arima_forecast_comparison(1, 2, 'GOOG')
plot_arima_forecast_comparison(2, 1, 'QCOM')
plot_arima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="ARIMA(1,1,1) MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **SARIMA (Seasonal ARIMA)**
SARIMA (Seasonal ARIMA) extends the ARIMA model to account for seasonality in time series data. The model includes both non-seasonal and seasonal elements:
(1−ϕ 
1
​
 B−⋯−ϕ 
p
​
 B 
p
 )(1−Φ 
1
​
 B 
s
 −⋯−Φ 
P
​
 B 
sP
 )(1−B) 
d
 (1−B 
s
 ) 
D
 X 
t
​
 =(1+θ 
1
​
 B+⋯+θ 
q
​
 B 
q
 )(1+Θ 
1
​
 B 
s
 +⋯+Θ 
Q
​
 B 
sQ
 )ϵ 
t
​


Here, 
B is the backshift operator, 
d and 
D are the orders of regular and seasonal differencing, 
p, 
q, 
P, and 
Q are the orders of the autoregressive and moving average terms for the non-seasonal and seasonal parts, and 
s is the length of the seasonal cycle.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Function to fit SARIMA model and forecast the next period
def forecast_sarima(stock_data):
    # Fit a SARIMA model for future prediction
    model_future = SARIMAX(stock_data, order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12]
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit a SARIMA model excluding the last data point for last period prediction
    model_last = SARIMAX(stock_data[:-1], order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12]
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_sarima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['SARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='SARIMA'].head()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_sarima_forecast_comparison(row, col, stock_name):
    # Fit SARIMA model excluding the last 12-weekly data
    model = SARIMAX(_12_weekly_averages[stock_name][:-1], order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12] model
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_sarima_forecast_comparison(1, 1, 'AMZN')
plot_sarima_forecast_comparison(1, 2, 'GOOG')
plot_sarima_forecast_comparison(2, 1, 'QCOM')
plot_sarima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="SARIMA(1,1,1)(1,1,1)[12] MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from pmdarima.arima import auto_arima

# Function to fit auto_arima model and forecast the next period
def forecast_auto_arima(stock_data):
    # Fit an auto_arima model for future prediction
    model_future = auto_arima(stock_data, seasonal=True, m=1, trace=False,
                              error_action='ignore', suppress_warnings=True,
                              stepwise=True)
    future_forecast = model_future.predict(n_periods=1)

    # Fit an auto_arima model excluding the last data point for last period prediction
    model_last = auto_arima(stock_data[:-1], seasonal=True, m=1, trace=False,
                            error_action='ignore', suppress_warnings=True,
                            stepwise=True)
    last_forecast = model_last.predict(n_periods=1)

    actual = stock_data[-1]
    return model_future, future_forecast[0], last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_auto_arima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['Auto-ARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='Auto-ARIMA'].head()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pmdarima.arima import auto_arima

# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_auto_arima_forecast_comparison(row, col, stock_name):
    # Fit auto_arima model excluding the last 12-weekly data
    model = auto_arima(_12_weekly_averages[stock_name][:-1], seasonal=True, m=1, trace=False,
                       error_action='ignore', suppress_warnings=True, stepwise=True)
    forecast = model.predict(n_periods=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_auto_arima_forecast_comparison(1, 1, 'AMZN')
plot_auto_arima_forecast_comparison(1, 2, 'GOOG')
plot_auto_arima_forecast_comparison(2, 1, 'QCOM')
plot_auto_arima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="Auto-ARIMA Model Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


In [ ]:
from darts import TimeSeries
from darts.models import ExponentialSmoothing

def forecast_auto_ets(stock_data, seasonal_period=4):
    # Convert to numpy array if needed
    values = stock_data.values if isinstance(stock_data, pd.Series) else np.array(stock_data)

    # Create a TimeSeries object from the values
    series = TimeSeries.from_values(values)

    # Check data length against twice the seasonal period for seasonal modeling
    if len(values) >= 2 * seasonal_period:
        # Enough data for seasonal modeling
        model = ExponentialSmoothing(seasonal_periods=seasonal_period)
        model_last = ExponentialSmoothing(seasonal_periods=seasonal_period)
    else:
        # Not enough data for seasonal modeling, use non-seasonal model
        print(f"Not enough data for seasonal modeling for series with length {len(values)} and seasonal period {seasonal_period}.")
        model = ExponentialSmoothing(seasonal_periods=None)
        model_last = ExponentialSmoothing(seasonal_periods=None)

    # Fit the models and make predictions
    model.fit(series)
    future_forecast = model.predict(n=1)
    
    # For last period prediction, create a TimeSeries object from all but the last value
    series_last = TimeSeries.from_values(values[:-1])
    model_last.fit(series_last)
    last_forecast = model_last.predict(n=1)

    actual = values[-1]
    return future_forecast.values()[0][0], last_forecast.values()[0][0], actual

# Example usage assuming '_12_weekly_averages' is a DataFrame with each column as a time series
models_future_forecasts = {}
models_last_forecasts = {}

for column in _12_weekly_averages.columns:
    future_forecast, last_forecast, actual = forecast_auto_ets(_12_weekly_averages[column])
    models_future_forecasts[column] = future_forecast
    models_last_forecasts[column] = [actual, last_forecast, actual - last_forecast]

    df = pd.DataFrame([['AutoETS', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates().reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val for key, val in models_future_forecasts.items()})
df_results[df_results['Model']=='AutoETS'].head()

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from lightgbm import LGBMRegressor

def train_lightgbm_forecast(series, lags, forecast_horizon=1):
    """
    Train a LightGBM model using Darts' RegressionModel wrapper and forecast the next time steps.
    """
    # Create a TimeSeries object if not already
    if not isinstance(series, TimeSeries):
        series = TimeSeries.from_series(series)

    # Define the LightGBM model
    model = LGBMRegressor()

    # Darts RegressionModel wrapper around LightGBM
    reg_model = RegressionModel(
        model=model,
        lags=lags,  # specify lags: this can be a list of lags or an integer
        output_chunk_length=forecast_horizon
    )

    # Fit the model
    reg_model.fit(series)

    # Perform prediction
    forecast = reg_model.predict(n=forecast_horizon)

    return reg_model, forecast

# Example usage with a time series data
models = {}
forecasts = {}

# Assuming '_12_weekly_averages' is a DataFrame with time series data in each column
for column in _12_weekly_averages.columns:
    series = TimeSeries.from_series(_12_weekly_averages[column])
    model, forecast = train_lightgbm_forecast(series, lags=12)  # Assuming using past 12 periods as features
    models[column] = model
    forecasts[column] = forecast
    
    # Storing results
    actual = _12_weekly_averages[column][-1]  # last known value
    predicted = forecast.values()[-1][0]  # last predicted value
    df = pd.DataFrame([['LightGBM', column, actual, predicted, actual - predicted]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.reset_index(drop=True)

# Display results
print('Forecast results using LightGBM:')
print(df_results)


In [ ]:
df_results.groupby('Model').agg({E})